# Intermediate SQL with SQLite in Python

In this lesson, we will work with the **Sakila** database—a sample database originally created by MySQL and ported to SQLite. The Sakila database contains a rich schema (including tables such as `actor`, `film`, `film_actor`, etc.) that provides real-world data to explore advanced SQL techniques.

### What You'll Learn:
- How to connect to a SQLite database file (`sakila.db`) using Python's `sqlite3` module.
- How to explore the Sakila database schema by listing tables and previewing data.
- How to write SQL queries using JOINs, aggregate functions, GROUP BY, and HAVING.
- How to apply these techniques with partner practice exercises.

**Note:** Ensure the `sakila.db` file is in your working directory before running the notebook.

In [8]:
import sqlite3
import pandas as pd

# Set the path to the SQLite database
db_path = "IN-CLASS\sakila.db"

# Connect to SQLite database
conn = sqlite3.connect(db_path)
print("Connected to SQLite Sakila database!")

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\sageo\AppData\Local\Temp\ipykernel_18504\2946681569.py:5: SyntaxWarning: invalid escape sequence '\s'
  db_path = "IN-CLASS\sakila.db"
C:\Users\sageo\AppData\Local\Temp\ipykernel_18504\2946681569.py:5: SyntaxWarning: invalid escape sequence '\s'
  db_path = "IN-CLASS\sakila.db"


OperationalError: unable to open database file

### Exploring the Database Schema

Before diving into queries, it's useful to know which tables are available in the Sakila database. We'll list all tables by querying the SQLite master table.

In [5]:
# List all tables in the database
tables = pd.read_sql("""SELECT name 
                        FROM sqlite_master 
                        WHERE type='table';""", conn)

print("Tables in the database:")
print(tables)

Tables in the database:
Empty DataFrame
Columns: [name]
Index: []


### Viewing Sample Data from the 'actor' Table

Let's preview a few rows from the `actor` table to see what kind of data it holds.

Select the following columns:
- actor_id
- first_name
- last_name

Connect two string columns with || ' ' || and create an alias with "AS"

In [3]:
# Define SQL query to fetch the first 5 rows from the actor table.
query_actor = """SELECT actor_id, first_name || ' ' || last_name AS actor_full_name
                 FROM actor 
                 LIMIT 5;"""

# Execute the query using pd.read_sql.
df_actor = pd.read_sql(query_actor, conn)

print("Sample rows from the 'actor' table:")
print(df_actor)

DatabaseError: Execution failed on sql 'SELECT actor_id, first_name || ' ' || last_name AS actor_full_name
                 FROM actor 
                 LIMIT 5;': no such table: actor

### Viewing Sample Data from the 'film' Table

Let's preview a few rows from the `film` table to see what kind of data it holds.

Select the following columns:
- film_id
- title
- length
- rating

In [17]:
# Define SQL query to fetch the first 5 rows from the actor table.
query_film = """SELECT film_id, title, length, rating
                 FROM film 
                 LIMIT 5;"""

# Execute the query using pd.read_sql.
df_film = pd.read_sql(query_film, conn)

print("Sample rows from the 'film' table:")
print(df_film)

Sample rows from the 'film' table:
   film_id             title  length rating
0        1  ACADEMY DINOSAUR      86     PG
1        2    ACE GOLDFINGER      48      G
2        3  ADAPTATION HOLES      50  NC-17
3        4  AFFAIR PREJUDICE     117      G
4        5       AFRICAN EGG     130      G


### Viewing Sample Data from the 'film_actor' Table

Let's preview a few rows from the `film_actor` table to see what kind of data it holds.

Select all columns.

In [18]:
# Define SQL query to fetch the first 5 rows from the actor table.
query_film_actor = """SELECT *
                    FROM film_actor 
                    LIMIT 5;"""

# Execute the query using pd.read_sql.
df_film_actor = pd.read_sql(query_film_actor, conn)

print("Sample rows from the 'film' table:")
print(df_film_actor)

Sample rows from the 'film' table:
   actor_id  film_id          last_update
0         1        1  2021-03-06 15:52:45
1         1       23  2021-03-06 15:52:45
2         1       25  2021-03-06 15:52:45
3         1      106  2021-03-06 15:52:45
4         1      140  2021-03-06 15:52:45


### JOIN Query: Actors and Their Films

Next, we'll join three tables—`actor`, `film_actor`, and `film`—to list film titles along with the full names of actors who appear in them.

- **Tables Involved:**
  - `actor`: Contains actor details.
  - `film_actor`: A bridge table linking actors to films.
  - `film`: Contains film details.

The join connects:
- `actor.actor_id` with `film_actor.actor_id`
- `film_actor.film_id` with `film.film_id`

Let's see the first 10 actor-film associations.

In [28]:
# Define SQL query for actor-film associations.
join_query = """SELECT a.first_name, a.last_name, f.title
                FROM actor AS a
                JOIN film_actor AS fa ON a.actor_id = fa.actor_id
                JOIN film AS f ON fa.film_id = f.film_id
                ORDER BY a.actor_id
                LIMIT 10;
            """

# Execute the query using pd.read_sql.
df_join = pd.read_sql(join_query, conn)

print("Actor - Film Associations (first 10 rows):")
print(df_join)

Actor - Film Associations (first 10 rows):
  first_name last_name                  title
0   PENELOPE   GUINESS       ACADEMY DINOSAUR
1   PENELOPE   GUINESS   ANACONDA CONFESSIONS
2   PENELOPE   GUINESS            ANGELS LIFE
3   PENELOPE   GUINESS  BULWORTH COMMANDMENTS
4   PENELOPE   GUINESS          CHEAPER CLYDE
5   PENELOPE   GUINESS     COLOR PHILADELPHIA
6   PENELOPE   GUINESS        ELEPHANT TROJAN
7   PENELOPE   GUINESS    GLEAMING JAWBREAKER
8   PENELOPE   GUINESS         HUMAN GRAFFITI
9   PENELOPE   GUINESS         KING EVOLUTION


### Aggregation Query: Counting Actors per Film

Now, we'll count the number of actors featured in each film. This query uses:
- **JOIN:** To connect `film` and `film_actor`.
- **GROUP BY:** To group data by film title.
- **HAVING:** To filter films with more than 3 actors.

This helps identify films with larger casts.

In [20]:
# Define SQL query for counting actors per film.
agg_query = """SELECT film.title, COUNT(film_actor.actor_id) AS actor_count
               FROM film
               JOIN film_actor ON film.film_id = film_actor.film_id
               GROUP BY film.title
               HAVING actor_count > 3
               ORDER BY actor_count DESC
               LIMIT 10;
               """

# Execute the query using pd.read_sql.
df_agg = pd.read_sql(agg_query, conn)

print("Films with more than 3 actors (top 10):")
print(df_agg)

Films with more than 3 actors (top 10):
               title  actor_count
0   LAMBS CINCINATTI           15
1          RANDOM GO           13
2    MUMMY CREATURES           13
3    DRACULA CRYSTAL           13
4         CRAZY HOME           13
5        CHITTY LOCK           13
6  BOONDOCK BALLROOM           13
7   TITANIC BOONDOCK           12
8        SKY MIRACLE           12
9    LONELY ELEPHANT           12


# Group Exercises
- Find a neighbor, yes, move around and make a new friend!
- Work through the following practice exercises together.

# Partner Practice Exercise

Now that we've seen several examples of JOINs and aggregation queries, it's time for you to practice with a partner. Complete the following tasks using the Sakila database:

1. **Task 1: Write a JOIN Query**
   - Write a query to retrieve a list of films along with the full names of the actors (first and last name) who acted in them.
   - *Hint:* Join the `film`, `film_actor`, and `actor` tables. Use string concatenation (e.g., `first_name || ' ' || last_name`) for full names.

2. **Task 2: Write an Aggregation Query**
   - Write a query to find the average film length for each film category.
   - *Hint:* Join the `film`, `film_category`, and `category` tables, then use the `AVG()` function with `GROUP BY`.

3. **Task 3: Challenge Task (Optional)**
   - Combine your results: Write a query to list film titles, the count of actors in each film, and the film category. Filter to show only films with more than 5 actors.
   - *Hint:* You might need to use multiple joins and group by both film title and category.

Discuss your approaches with your partner, compare your results, and be ready to review a few solutions as a class.

In [ ]:
# Task 1: Retrieve a list of films along with the full names of the actors.
task1_query = """
SELECT f.title,
       a.first_name || ' ' || a.last_name AS actor_full_name
FROM film AS f
JOIN film_actor AS fa ON f.film_id = fa.film_id
JOIN actor AS a ON fa.actor_id = a.actor_id
ORDER BY f.title, a.last_name, a.first_name
LIMIT 20;
"""

# Execute the query using pd.read_sql.
df_task1 = pd.read_sql(task1_query, conn)

print("Task 1: Films with Actor Full Names (first 20 rows):")
print(df_task1)

Task 1: Films with Actor Full Names (first 20 rows):
               title   actor_full_name
0   ACADEMY DINOSAUR       JOHNNY CAGE
1   ACADEMY DINOSAUR      ROCK DUKAKIS
2   ACADEMY DINOSAUR   CHRISTIAN GABLE
3   ACADEMY DINOSAUR  PENELOPE GUINESS
4   ACADEMY DINOSAUR       MARY KEITEL
5   ACADEMY DINOSAUR      OPRAH KILMER
6   ACADEMY DINOSAUR      WARREN NOLTE
7   ACADEMY DINOSAUR       SANDRA PECK
8   ACADEMY DINOSAUR       MENA TEMPLE
9   ACADEMY DINOSAUR     LUCILLE TRACY
10    ACE GOLDFINGER        CHRIS DEPP
11    ACE GOLDFINGER       BOB FAWCETT
12    ACE GOLDFINGER      SEAN GUINESS
13    ACE GOLDFINGER  MINNIE ZELLWEGER
14  ADAPTATION HOLES    JULIANNE DENCH
15  ADAPTATION HOLES       BOB FAWCETT
16  ADAPTATION HOLES     RAY JOHANSSON
17  ADAPTATION HOLES    CAMERON STREEP
18  ADAPTATION HOLES     NICK WAHLBERG
19  AFFAIR PREJUDICE    SCARLETT DAMON


In [22]:
# Task 2: Find the average film length for each film category.
task2_query = """
SELECT c.name AS category_name,
       AVG(f.length) AS avg_length
FROM film AS f
JOIN film_category AS fc ON f.film_id = fc.film_id
JOIN category AS c ON fc.category_id = c.category_id
GROUP BY c.name
ORDER BY avg_length DESC;
"""

# Execute the query using pd.read_sql.
df_task2 = pd.read_sql(task2_query, conn)

print("Task 2: Average Film Length per Category:")
print(df_task2)

Task 2: Average Film Length per Category:
   category_name  avg_length
0         Sports  128.202703
1          Games  127.836066
2        Foreign  121.698630
3          Drama  120.838710
4         Comedy  115.827586
5         Family  114.782609
6          Music  113.647059
7         Travel  113.315789
8         Horror  112.482143
9       Classics  111.666667
10        Action  111.609375
11           New  111.126984
12     Animation  111.015152
13      Children  109.800000
14   Documentary  108.750000
15        Sci-Fi  108.196721


In [23]:
# Task 3: List film titles, count of actors, and film category for films with more than 5 actors.
task3_query = """
SELECT f.title,
       COUNT(DISTINCT fa.actor_id) AS actor_count,
       c.name AS category_name
FROM film AS f
JOIN film_actor AS fa ON f.film_id = fa.film_id
JOIN film_category AS fc ON f.film_id = fc.film_id
JOIN category AS c ON fc.category_id = c.category_id
GROUP BY f.title, c.name
HAVING actor_count > 5
ORDER BY actor_count DESC;
"""

# Execute the query using pd.read_sql.
df_task3 = pd.read_sql(task3_query, conn)

print("Task 3: Films with more than 5 Actors (showing film title, actor count, category):")
print(df_task3)

Task 3: Films with more than 5 Actors (showing film title, actor count, category):
                 title  actor_count category_name
0     LAMBS CINCINATTI           15         Games
1    BOONDOCK BALLROOM           13        Travel
2          CHITTY LOCK           13         Drama
3           CRAZY HOME           13        Comedy
4      DRACULA CRYSTAL           13      Classics
..                 ...          ...           ...
451          WIFE TURN            6   Documentary
452      WOLVES DESIRE            6        Travel
453       WORDS HUNTER            6         Music
454      WYOMING STORM            6           New
455       ZHIVAGO CORE            6        Horror

[456 rows x 3 columns]


## Close the connection to the database.
Always close the connection when you're done to free up resources.

In [24]:
# Close the connection
#conn.close()